## Creating your first LangChain project

In [3]:
from langchain_aws import ChatBedrock
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [5]:
chat = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

In [6]:
messages = [
    (
        "system",
        "You are a helpful AI that helps the user make travel plans. Respond only in a single line.",
    ),
    ("human", "I want to go skiing. Which city should I go to?"),
]
first_msg = chat.invoke(messages)

AIMessage(content='For a great skiing destination, I would recommend cities like Aspen, Colorado; Park City, Utah; or Whistler, British Columbia.', additional_kwargs={'usage': {'prompt_tokens': 41, 'completion_tokens': 33, 'total_tokens': 74}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 41, 'completion_tokens': 33, 'total_tokens': 74}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-f7095175-e97e-4401-8a65-c916921c7c32-0', usage_metadata={'input_tokens': 41, 'output_tokens': 33, 'total_tokens': 74})

In [ ]:
print(first_msg.content)

In [ ]:
print(first_msg.usage_metadata["total_tokens"])

#### Google Generative AI API

In [ ]:
%pip install langchain-google-genai

In [ ]:
#os.environ['GOOGLE_API_KEY'] = ""

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful AI that helps the user make travel plans. Respond only in a single line.",
    ),
    ("human", "I want to go skiing. Which city should I go to?"),
]
first_msg = chat.invoke(messages)
first_msg

In [ ]:
print(first_msg.content)

## Chat vs LLM in Langchain

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI()
city = llm.predict("I want to go skiing. Which city should I go to?")
print(city)

In [ ]:
places = llm.predict("What else can I do in that city?")
print(places)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful AI that helps the user make travel plans. Respond only in ble line.",
    ),
    ("human", "I want to go skiing. Which city should I go to?"),
    ("ai", "You should consider visiting Aspen, Colorado for a great skiing experience."),
    ("human", "What else can I do in that city?"),
]
ai_msg = chat.invoke(messages)
print(ai_msg.content)

### Model parameters

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

#### Image generation using Dall-E

In [ ]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

In [ ]:
image_url = DallEAPIWrapper().run("Generate a high resolution image of a cute dog")
image_url

For other tools refer: https://python.langchain.com/v0.2/docs/integrations/tools/

## Prompt Templates

In [ ]:
from langchain import PromptTemplate

dictionary_template = PromptTemplate(
    input_variables =['word'],
    template = "Give me a one line definition of {word}. Then, give one example of how it is used."
)
p = dictionary_template.format(word="Sesquipedalian")
print(p)


In [ ]:
openAILLM = ChatOpenAI()
print(openAILLM.invoke(p).content)

#### Multiple input variables

In [ ]:
from langchain import PromptTemplate

define_translate = PromptTemplate(
    input_variables =['word','language'],
    template = "Give me a one line definition of {word}. Then, give the translation of that word in {language}."
)
p = define_translate.format(word="Sesquipedalian", language="Hindi")
print(p)
print(chat.invoke(p).content)

#### Taking input from user

In [ ]:
import tkinter as tk
from tkinter import simpledialog

ROOT = tk.Tk()
ROOT.withdraw()
# the input dialog
user_input = simpledialog.askstring(title="Quick Dictionary",
                                  prompt="Enter the word you want to learn about:")

dictionary_template = PromptTemplate(
    input_variables =['word'],
    template = "Give me a one line definition of {word}. Then, give one example of how it is used."
)
p = dictionary_template.format(word=user_input)
print(chat.invoke(p).content)

#### General purpose prompt template

In [ ]:
template = """You are a helpful assistant.

Human: {human_input}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["human_input"], template=template
)

#### ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line definition of the word entered by user"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="Sesquipedalian")
messages

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line answer to user query"),
        ("human", "Who created theory of relativity?"),
        ("ai", "Albert Einstein developed the theory of relativity."),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="When was it created?")
messages

In [ ]:
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a helpful assistant that gives a one-line definition of the word entered by user"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)
messages = chat_template.format_messages(input="Callous")
messages

#### Few shot prompt templates

In [ ]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "rev": "I love this product",
        "answer": "positive"
    }, {
        "rev": "It was an average experience",
        "answer": "neutral"
    },{
        "rev": "I wonder why it is so highly rated.",
        "answer": "negative"
    }
]


# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["rev", "answer"],
    template= """
User: {rev}
AI: {answer}
"""
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = "Identify the sentiment of the user review. Here are some examples: "
# and the suffix our user input and output indicator
suffix = """
User: {review}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["review"],
    example_separator="\n"
)

user_review = "Well structured. Five stars."

print(few_shot_prompt_template.format(review=user_review))

## Chains

#### Generic chain - LLMChain

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Then, give one example of how it is used."
)

llm = ChatOpenAI()

#p = dictionary_template.format(word="Sesquipedalian")
#print(chat.invoke(p).content)

chain = LLMChain(llm=llm, prompt=dictionary_template)
chain.invoke("Anachronism")


In [ ]:
input_list = [
    {"word": "Sesquipedalian"},
    {"word": "Anachronism"},
    {"word": "Onomatopoeia"}
]

chain.apply(input_list)

In [ ]:
chain.generate(input_list)

#### Utility chains

In [ ]:
from langchain.chains import LLMMathChain

In [ ]:
#new code
calculator = LLMMathChain.from_llm(llm, verbose=True)
calculator.invoke("Calculate (625 raised to power (0.225))-(log10(100))")

In [ ]:
print(calculator.prompt.template)

In [ ]:
calc_llm =ChatOpenAI()
messages = [
    ("human", "Calculate (625 raised to power (0.225))-(log10(100))"),
]
calculation = calc_llm.invoke(messages)
print(calculation.content)

##### List of all chains - https://python.langchain.com/v0.1/docs/modules/chains/

### Sequential Chain

In [ ]:
marketing_text = PromptTemplate(
    input_variables =['name','description'],
    template = "Generate a one line facebook ad copy for a product called {name}. Below is a description of this product: {description}"
)

text_llm = ChatOpenAI()

text_chain = LLMChain(llm=text_llm, prompt=marketing_text, output_key="copy_text")


translate_text = PromptTemplate(input_variables=["copy_text"], template="""Translate this text to Hindi:

{copy_text} """)

from langchain_google_genai import ChatGoogleGenerativeAI

translate_llm = ChatGoogleGenerativeAI(model="gemini-pro")

translate_chain = LLMChain(llm=translate_llm, prompt=translate_text, output_key="translated_copy")

from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains = [text_chain, translate_chain],
    input_variables = ["name", "description"],
    output_variables = ["copy_text","translated_copy"]
)

seq_chain({"name": "AeroGlow Nightlight","description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling."})

### LCEL - Pipe operator and Runnables

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word - {word} itself, only the meaning."
)

llm = ChatOpenAI()

chain = dictionary_template | llm | StrOutputParser()

In [ ]:
chain.invoke({"word": "Onomatopoeia"})

In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

guesser_template = PromptTemplate(
    input_variables =["definition"],
    template = "Give me an English word which has the following meaning- {definition}."
)

llm = ChatOpenAI()

chain2 = guesser_template.pipe(llm).pipe(StrOutputParser())
chain2.invoke("A word that imitates the sound it represents.")

In [ ]:
seq_chain = {"definition": chain} | chain2

seq_chain.invoke({"word": "Sesquipedalian"})

#### RunnablePassthrough, RunnableLambda and RunnableParallel

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [ ]:
chain = RunnablePassthrough()

In [ ]:
chain.invoke("abcd")

In [ ]:
def output_length(input: str):
    output = len(input)
    return output

chain = RunnableLambda(output_length)
chain.invoke("input to output")

In [ ]:
chain = RunnableParallel(text = RunnablePassthrough(), length = RunnableLambda(output_length))
chain.invoke("start-tech academy")

#### Example

In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

explainer_template = PromptTemplate(
    input_variables =["topic"],
    template = "Explain this topic in a single paragraph as if you were explaining it to a 10 year old - {topic}."
)

openAIllm = ChatOpenAI()

OpenAI_chain = explainer_template | openAIllm | StrOutputParser()
OpenAI_chain.invoke({"topic": "Gravity"})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

googlellm = ChatGoogleGenerativeAI(model="gemini-pro")

google_chain = explainer_template | googlellm | StrOutputParser()
google_chain.invoke({"topic": "Gravity"})

In [ ]:
analyzer_template = PromptTemplate(
    input_variables =["topic", "explanation1", "explanation2"],
    template = """
    Which of the two explanations given below are better for explaining {topic} to 10 year old students.
    ###
    Explanation 1 - {explanation1}
    ###
    ***
    Explanation 2 - {explanation2}
    ***
    """
)

combined_chain = (
    RunnableParallel({"explanation1": OpenAI_chain, "explanation2": google_chain, "topic": RunnablePassthrough()})
    | analyzer_template | openAIllm | StrOutputParser()
)

combined_chain.invoke({"topic": "Gravity"})

In [ ]:
analyzer_chain = analyzer_template | googlellm | StrOutputParser()

updated_chain = (
    RunnableParallel({"explanation1": OpenAI_chain, "explanation2": google_chain, "topic": RunnablePassthrough()}) 
    | RunnableParallel({"input": RunnablePassthrough(), "analysis": analyzer_chain})
)

updated_chain.invoke({"topic": "Gravity"})

#### Dynamic Routing

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

classifier_template = PromptTemplate(
    input_variables = ["question", "answer"],
    template = """You are given a question and the user's response to that question. Classify the response as either `Correct`, or `Incorrect`.
    Do not respond with more than one word.
    Question - {question}
    User's Answer - {answer}
    Classification:"""
)

llm = ChatOpenAI()

output_parser = StrOutputParser()

classifier_chain = classifier_template | llm | output_parser

classifier_chain.invoke({"question": "what are penguins", "answer": "Penguins are birds"})

In [ ]:
correct_template = PromptTemplate(
    input_variables = ["question"],
    template = """The user was asked the following question and user answered it correctly. Now ask a more difficult question on the same topic to the user.
    Question: {question}
    New Question:"""
)

correct_chain = correct_template | llm | output_parser

incorrect_template = PromptTemplate(
    input_variables = ["question"],
    template = """The user was asked the following question and user answered it incorrectly. Give the correct answer and explain it to the user.
    Question: {question}
    Correct Answer:
    Explanation: """
)

incorrect_chain = incorrect_template | llm | output_parser

In [ ]:
def route(info):
    if  info["result"].lower() == "correct":
        return correct_chain
    elif info["result"].lower() == "incorrect":
        return incorrect_chain
    else:
        return "Format is not correct"

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough


final_chain = RunnableParallel({"result": classifier_chain, "question": lambda x: x["question"], "answer": lambda x: x["answer"]}) | RunnableParallel({"response":RunnableLambda(route), "input": RunnablePassthrough()})

In [ ]:
final_chain.invoke({"question": "what are penguins", "answer": "Penguins are sea animals"})

In [ ]:
final_chain.invoke({"question": "what are SQL joins", "answer": "Joins are used to join two queries"})

In [ ]:
final_chain.invoke({"question": "what are SQL joins", "answer": "SQL joins are operations used to combine rows from two or more tables based on a related column"})

### Output Parsing

#### StrOutputParser()

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word itself, only the meaning."
)

llm = ChatOpenAI()

chain = dictionary_template | llm 
chain.invoke("Onomatopoeia")

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word itself, only the meaning."
)

llm = ChatOpenAI()

chain = dictionary_template | llm | StrOutputParser()
chain.invoke("Onomatopoeia")

#### StructuredOutputParser - Output as a specified Schema

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

response_schemas = [
    ResponseSchema(name="word", description="word entered by user"),
    ResponseSchema(name="meaning", description="One line meaning of the word given by user"),
    ResponseSchema(name="example", description="An example of how that word can be used in a line"),
    ResponseSchema(name="Etymology", description="Origin or history of the word")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
prompt = PromptTemplate(
    template="Provide the meaning, an example of how the word is used in a sentence and the etymology of this word: {word}.\n{format_instructions}",
    input_variables=["word"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | chat_model | output_parser

chain.invoke("Onomatopoeia")

#### Comma separated list

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {topic}.\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI()

chain = prompt | model | output_parser 

In [ ]:
print(format_instructions)

In [ ]:
chain.invoke({"topic": "healthy foods"})

#### Date Time Parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [ ]:
prompt

In [ ]:
chain = prompt | chat_model | output_parser

In [ ]:
output = chain.invoke({"question": "When did India get independence?"})
print(output)

Output Parser Documentation - https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/quick_start/

## Memory

In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prompt_template = PromptTemplate(
    input_variables =["history", "new_input"],
    template = """You are having a chat with a human.
    previous chat: {history}
    Human: {new_input}
    Assistant: """
)

memory = ConversationBufferMemory()

#llm = ChatOpenAI()
llm = OpenAI()

chain = LLMChain(llm=llm, prompt=prompt_template, memory = memory, verbose=True)

chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})


In [ ]:
chain.invoke({"new_input": "What is the total value of cotton exports of this country"})

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with a human."
        ), 
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),  
    ]
)

chat_memory = ConversationBufferMemory(return_messages=True)

chat_llm = ChatOpenAI()

chat_chain = LLMChain(llm=chat_llm, prompt=chat_prompt, memory = chat_memory, verbose=True)

chat_chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})

In [ ]:
chat_chain.invoke({"new_input": "What is the total value of cotton exports of this country"})

#### Changing the memory variable name

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with a human."
        ), 
        MessagesPlaceholder(
            variable_name="prev_conv"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),  
    ]
)

chat_memory = ConversationBufferMemory(memory_key="prev_conv", return_messages=True)

chat_llm = ChatOpenAI()

chat_chain = LLMChain(llm=chat_llm, prompt=chat_prompt, memory = chat_memory, verbose=True)

chat_chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})

#### Adding messages to memory

In [ ]:
sample_memory = ConversationBufferMemory(memory_key="history")

In [ ]:
sample_memory.load_memory_variables({})

In [ ]:
sample_memory.chat_memory.add_user_message("Which country is the biggest exporter of cotton?")
sample_memory.chat_memory.add_ai_message("China")

In [ ]:
sample_memory.load_memory_variables({})

In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prompt_template = PromptTemplate(
    input_variables =["history", "new_input"],
    template = """You are having a chat with a human.
    previous chat: {history}
    Human: {new_input}
    Assistant: """
)


llm = ChatOpenAI()

chain = LLMChain(llm=llm, prompt=prompt_template, memory = sample_memory, verbose=True)


In [ ]:
chain.invoke({"new_input": "What is the total value of cotton exports of this country"})

#### Conversation Chain instead of LLMChain

In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationChain

memory = ConversationBufferMemory(memory_key="history")

llm = ChatOpenAI()

chain = ConversationChain(llm=llm, memory = memory, verbose=True)

chain.invoke({"input": "Which country is the biggest exporter of cotton?"})

In [ ]:
print(chain.prompt.template)

In [ ]:
chain.invoke({"input": "What is the total value of cotton exports of this country"})

In [ ]:
print(chain.memory.buffer)

#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)

llm = ChatOpenAI()

convo_chain = ConversationChain(llm=llm, memory=memory)

convo_chain.invoke({"input": "Which country is the biggest exporter of cotton?"})

In [ ]:
convo_chain.invoke({"input": "What is the total value of cotton exports of this country"})

In [ ]:
print(convo_chain.memory.buffer)

#### ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "Which country is the biggest exporter of cotton?"}, {"output": "China"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.invoke(input="Which country is the biggest exporter of cotton?")

In [ ]:
conversation_with_summary.invoke(input="What is the total value of cotton exports of this country")

#### Runnable with Message History with session ID

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are having a chat with a human.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


runnable = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
runnable.invoke(
    {"input": "Which country is the biggest exporter of cotton?"},
    config={"configurable": {"session_id": "sess1"}},
)

In [ ]:
# Remembers
runnable.invoke(
    {"input": "What is the total value of cotton exports of this country"},
    config={"configurable": {"session_id": "sess1"}},
)

In [ ]:
# New session_id --> does not remember.
runnable.invoke(
    {"input": "What is the total value of cotton exports of this country"},
    config={"configurable": {"session_id": "sess2"}},
)

# RAG

### Document Loading

#### Text

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./RAGfiles/LangchainRetrieval.txt")
loader.load()

In [ ]:
pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./RAGfiles/Excel Course Document.pdf")
pages = loader.load_and_split()

In [ ]:
pages[1]

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('./RAGfiles/', glob="**/*.txt")

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
docs[1]

In [ ]:
loader = DirectoryLoader('./RAGfiles/', glob="**/*.txt", show_progress=True)
docs = loader.load()

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='./RAGfiles/Movie_collection_dataset.csv')
data = loader.load()

In [ ]:
print(data)

In [ ]:
loader = CSVLoader(file_path='./RAGfiles/Movie_collection_dataset.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['Genre', 'Budget', 'Actor_rating']
})

data = loader.load()

### Splitting the document - Chunking

#### Recursively split by character

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./RAGfiles/LangchainRetrieval.txt")
text = loader.load()

In [ ]:
text

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

In [ ]:
texts = text_splitter.split_documents(text)
print(texts[0])
print(texts[1])
print(texts[2])

### Embedding

#### OpenAI embedding

In [ ]:
pip install langchain-openai

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi",
        "What's up!",
        "Learning LangChain",
        "You should learn it from Start-Tech Academy"
    ]
)
len(embeddings), len(embeddings[0])

In [ ]:
embeddings[0]

In [ ]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

#### Huggingface embeddings

In [ ]:
pip install -U sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
embeddings_model = HuggingFaceInstructEmbeddings()

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

### Vector Storage

#### Chroma

In [ ]:
pip install langchain-chroma

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("./RAGfiles/LangchainRetrieval.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [ ]:
query = "What is text embedding and how does langchain help in doing it"
docs = db.similarity_search(query)
print(docs[1].page_content)

In [ ]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

In [ ]:
pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("./RAGfiles/LangchainRetrieval.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

## Retrievers

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("./RAGfiles/LangchainRetrieval.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [ ]:
retriever = db.as_retriever()

In [ ]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [ ]:
len(docs)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("What is text embedding and how does langchain help in doing it")


In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 1})

In [ ]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [ ]:
docs

In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)

In [ ]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [ ]:
docs

### Tools

#### Creating own custom tool

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
@tool
def name_of_tool(input: str) -> str:
    """Tool_Description"""
    return "Result"

In [ ]:
print(name_of_tool.name)
print(name_of_tool.description)
print(name_of_tool.args)

#### Defining, Binding and Calling the tool

In [ ]:
@tool
def sta_coins(input: float) -> float:
    """Use this tool to convert USD to Start-Tech Academy coins"""
    return 1.3*(float(input))

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm_bind_tools = llm.bind_tools([sta_coins])

In [ ]:
result = llm_bind_tools.invoke("How many start-tech academy coins can I get for USD10")
result

In [ ]:
result.tool_calls

In [ ]:
tool_mapping = {
    'sta_coins': sta_coins
}
tool_mapping

In [ ]:
tool = tool_mapping[result.tool_calls[0]["name"]]

In [ ]:
tool_output = tool.invoke(result.tool_calls[0]["args"])

In [ ]:
tool_output

In [ ]:
# For multiple tool calls in LLM response, we can loop through the list of tools in tool call
tool_mapping = {
    'sta_coins': sta_coins
    'tool2_name': tool2
}
tool_mapping
for tool_call in result.tool_calls:
    tool = tool_mapping[tool_call["name"]]
    tool_output = tool.invoke(tool_call["args"])

### Using in-built tools
https://python.langchain.com/v0.1/docs/integrations/tools/

#### DuckDuckGo Search tool

In [ ]:
%pip install --upgrade --quiet  duckduckgo-search

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [ ]:
search.name, search.description

In [ ]:
search.run("What are tools in Langchain?")

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
search.invoke("What are tools in Langchain?")

### Wikipedia

In [ ]:
%pip install --upgrade --quiet  wikipedia

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.invoke("LangChain")

### Agents

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. For answering the user query, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [search, wikipedia]

llm = ChatOpenAI()



In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "weather in delhi"})

In [ ]:
agent_executor.invoke({"input": "When was Nelson Mandela born and what are some of his famous quotes?"})

#### Agent with memory

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Based on user query and the chat history, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("placeholder", "{history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
agent_with_history.invoke(
    {"input": "When was Nelson Mandela born?"},
    config={"configurable": {"session_id": "sess1"}},
)

In [ ]:
result = agent_with_history.invoke(
    {"input": "What are some of his famous quotes?"},
    config={"configurable": {"session_id": "sess1"}},
)

In [ ]:
result['output']

### LangSmith for monitoring the application

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['LANGCHAIN_PROJECT'] = "langchain_course"

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("what is langchain")

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Based on user query and the chat history, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("placeholder", "{history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

agent_with_history.invoke(
    {"input": "When was Nelson Mandela born?"},
    config={"configurable": {"session_id": "sess1"}},
)